# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [687]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url="https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url=root_url+"/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req=requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)

if req.status_code==200:
    print(req.text)
else:
    print(req.status_code)

"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [626]:
# Set the countries_url variable (1 line)
countries_url=root_url+"/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req=requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries=req.json()

# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)


403 {'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [ ]:
# Set the cookie_url variable (1 line)
cookie_url=root_url+"/cookie"
# Query the enpoint, set the cookies variable and display it (2 lines)
r=requests.get(cookie_url)
cookies=r.cookies
print(r.cookies)


<RequestsCookieJar[<Cookie user_cookie=a190b04f-0f05-44e2-8447-c0af49122799 for country-leaders.onrender.com/>]>


In [696]:
def refresh_cookies():
    cookie_url=root_url+"/cookie"
    r=requests.get(cookie_url)
    cookies=r.cookies
    print(r.cookies) 
refresh_cookies()
    

<RequestsCookieJar[<Cookie user_cookie=524a2175-2a6f-444a-9cc4-b6a506f6dedd for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [628]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries=requests.get(countries_url, cookies=cookies).json()

# display the countries variable (1 line)
print(countries)

['fr', 'be', 'ma', 'us', 'ru']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [629]:
# Set the leaders_url variable (1 line)
leaders_url= root_url+"/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders=requests.get(leaders_url, cookies=cookies).json()
# display the leaders variable (1 line)
print(leaders)

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [ ]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
params = {"country": "fr"}
leaders=requests.get(leaders_url, cookies=cookies, params=params).json()
# assign the output to the leaders variable (1 line)
# display the leaders variable (1 line)
print(leaders)

[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': 'h

In [631]:
len(leaders)

31

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [632]:
# 4 lines
leaders_per_country={}
for c in countries:
    params = {"country": c}
    leader=requests.get(leaders_url, cookies=cookies, params=params).json()
    leaders_per_country[c]=leader
print(leaders_per_country)


{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_u

In [633]:
# or 1 line
len(leaders)


31

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [634]:
def get_leaders():
    #root_url
    r=requests.get(root_url+"/cookie")
    cookies=r.cookies
    countries=requests.get(root_url+"/countries", cookies=cookies).json()
    for c in countries:
        params = {"country": c}
        leader=requests.get(leaders_url, cookies=cookies, params=params).json()
        leaders_per_country[c]=leader
    print(leaders_per_country)

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [635]:
# 2 lines
get_leaders()

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_u

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [636]:
# 3 lines
leader1=leaders[3]
url=leader1["wikipedia_url"]
print(url)

https://fr.wikipedia.org/wiki/Charles_de_Gaulle


Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [637]:
from bs4 import BeautifulSoup
a=requests.get(url)
content=a.content
soup=BeautifulSoup(content, "html")
soup.prettify()
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   Charles de Gaulle — Wikipédia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-conte

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [638]:
# 2 lines
paragraph= soup.find_all("p")
paragraph1 = paragraph[1]
print(paragraph1.text) 

Général de Gaulle


If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [639]:
# <10 lines
first_paragraph=[]
for p in paragraph:
    if p.find("b")!=None:
        p_index=paragraph.index(p)
        first_paragraph.append(paragraph[p_index].get_text())
        break
print(first_paragraph)

["Charles de Gaulle (/ʃaʁl də ɡol/[n 3] Écouterⓘ), né le 22 novembre 1890 à Lille (Nord) et mort le 9 novembre 1970 à Colombey-les-Deux-Églises (Haute-Marne), est un militaire, résistant, homme d'État et écrivain français. Pendant la Seconde Guerre mondiale, le général de Gaulle a fondé la France libre\xa0; à la Libération, il préside le gouvernement provisoire jusqu'en 1946. Instigateur de la Ve\xa0République, il en a été le premier président du 8 janvier 1959 au 28 avril 1969.\n"]


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [640]:
# 10 lines

def get_first_paragraph(wikipedia_url):
       print(wikipedia_url) # keep this for the rest of the notebook

       a=requests.get(wikipedia_url)
       content=a.content
       soup=BeautifulSoup(content, "html")

       paragraph= soup.find_all("p")
       first_paragraph=[]
       for p in paragraph:
              if p.find("b") != None:
                     p_index=paragraph.index(p)
                     first_paragraph.append(paragraph[p_index].get_text())
                     break
       print(first_paragraph)
#   return first_paragraph

In [641]:
# Test: 3 lines
get_first_paragraph(url)    # testing the function

https://fr.wikipedia.org/wiki/Charles_de_Gaulle
["Charles de Gaulle (/ʃaʁl də ɡol/[n 3] Écouterⓘ), né le 22 novembre 1890 à Lille (Nord) et mort le 9 novembre 1970 à Colombey-les-Deux-Églises (Haute-Marne), est un militaire, résistant, homme d'État et écrivain français. Pendant la Seconde Guerre mondiale, le général de Gaulle a fondé la France libre\xa0; à la Libération, il préside le gouvernement provisoire jusqu'en 1946. Instigateur de la Ve\xa0République, il en a été le premier président du 8 janvier 1959 au 28 avril 1969.\n"]


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [642]:
import re
pattern = re.compile(r"\[|\]|//|/xa0|\b\w+\s*ⓘ")
first_paragraph_cleaned = re.sub(pattern, "", str(first_paragraph))
print(first_paragraph_cleaned)

"Charles de Gaulle (/ʃaʁl də ɡol/n 3 ), né le 22 novembre 1890 à Lille (Nord) et mort le 9 novembre 1970 à Colombey-les-Deux-Églises (Haute-Marne), est un militaire, résistant, homme d'État et écrivain français. Pendant la Seconde Guerre mondiale, le général de Gaulle a fondé la France libre\xa0; à la Libération, il préside le gouvernement provisoire jusqu'en 1946. Instigateur de la Ve\xa0République, il en a été le premier président du 8 janvier 1959 au 28 avril 1969.\n"


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [643]:
# 10 lines
def get_first_paragraph(wikipedia_url):
       #print(wikipedia_url) # keep this for the rest of the notebook

       a=requests.get(wikipedia_url)
       content=a.content
       soup=BeautifulSoup(content, "html")

       paragraph= soup.find_all("p")
       first_paragraph_raw=[]
       for p in paragraph:
            if p.find("b") != None:
                p_index=paragraph.index(p)
                first_paragraph_raw.append(paragraph[p_index].get_text())
                pattern = re.compile(r"\[|\]|//|/xa0|\b\w+\s*ⓘ")
                first_paragraph = re.sub(pattern, "", str(first_paragraph_raw))
                print(first_paragraph)
                break
       


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [644]:
# < 20 lines
get_first_paragraph(url)

"Charles de Gaulle (/ʃaʁl də ɡol/n 3 ), né le 22 novembre 1890 à Lille (Nord) et mort le 9 novembre 1970 à Colombey-les-Deux-Églises (Haute-Marne), est un militaire, résistant, homme d'État et écrivain français. Pendant la Seconde Guerre mondiale, le général de Gaulle a fondé la France libre\xa0; à la Libération, il préside le gouvernement provisoire jusqu'en 1946. Instigateur de la Ve\xa0République, il en a été le premier président du 8 janvier 1959 au 28 avril 1969.\n"


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [645]:
def get_first_paragraph(wikipedia_url):
       #print(wikipedia_url) # keep this for the rest of the notebook

    a=requests.get(wikipedia_url)
    content=a.content
    soup=BeautifulSoup(content, "html")

    paragraph= soup.find_all("p")
    first_paragraph_raw=[]
    for p in paragraph:
        if p.find("b") != None:
            p_index=paragraph.index(p)
            first_paragraph_raw.append(paragraph[p_index].get_text())
            pattern = re.compile(r"\[|\]|//|/xa0|\b\w+\s*ⓘ")
            first_paragraph = re.sub(pattern, "", str(first_paragraph_raw))
            return (first_paragraph)
            break


In [697]:
def get_leaders():
    #root_url
    r=requests.get(root_url+"/cookie")
    cookies=r.cookies
    leaders_url= root_url+"/leaders"
    countries=requests.get(root_url+"/countries", cookies=cookies).json()

    #print(countries, len(countries))
    leaders_per_country={}
    wikipedia_url=[]
    for country in countries:
        params = {"country": c}
        leaders=requests.get(leaders_url, cookies=cookies, params=params).json()
        leaders_per_country[country]=leaders
    #print(leaders_per_country.keys())
    for leader in leaders_per_country.keys():
        l=leaders_per_country[leader]
        #print(type(l), len(l))
        for leader1 in leaders_per_country[leader]:
            #print(leader1)
            a=leader1["wikipedia_url"]
            #print(get_first_paragraph(a))
            leader1["leaders_intro"]=str(get_first_paragraph(a))
            #print(leader1)
    print(leaders_per_country)
                 
get_leaders()

{'fr': [{'id': 'Q7747', 'first_name': 'Vladimir', 'last_name': 'Putin', 'birth_date': '1952-10-07', 'death_date': None, 'place_of_birth': 'Saint Petersburg', 'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87', 'start_mandate': '2000-05-07', 'end_mandate': '2008-05-07', 'leaders_intro': "'Влади́мир Влади́мирович Пу́тин (род. 7 октября 1952, Ленинград, СССР)\\xa0— российский государственный и политический деятель. Действующий президент Российской Федерации, председатель Государственного Совета Российской Федерации и Совета Безопасности Российской Федерации; Верховный главнокомандующий Вооружёнными силами Российской Федерации с 7 мая 2012\\xa0года. Ранее занимал должность президента с 7 мая 2000\\xa0по 7 мая 2008\\xa0года, также в 1999—2000 и 2008—2012\\xa0годах занимал должность председателя правительства Российской Федерации. Фактически 

In [647]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [648]:
# < 25 lines



Check the output of your function again.

In [649]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [650]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [651]:
# <25 lines


Test your new functions.



## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [652]:
# 3 lines
import json
try:
    with open("leaders.json", "w") as leaders_file:
        json.dump(leaders_per_country, leaders_file)
    print("done")
except Exception as e:
    print(f"file cant be saved: {e}")


done


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [653]:
# 3 lines
with open("leaders.json", "r") as leaders_file:
    content=json.load(leaders_file)
    print(content)

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_u

Make a function `save(leaders_per_country)` to call this code easily.

In [654]:
def save (leaders_per_country):
    try:
        with open("leaders.json", "w") as leaders_file:
            json.dump(leaders_per_country, leaders_file)
        print("done")
    except Exception as e:
         print(f"file cant be saved: {e}")

In [655]:
# Call the function (1 line)
save (leaders_per_country)

done


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!